# CV Model Comparison Results

In [ ]:
import os
import platform
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from matplotlib import font_manager as fm

import vis_utils

font_path = r'C:\Users\Felix\AppData\Local\Microsoft\Windows\Fonts\SourceSansPro-Regular.ttf'
fm.fontManager.addfont(font_path)
source_sans_pro = fm.FontProperties(fname=font_path)

plt.rcParams['font.family'] = source_sans_pro.get_name()

In [ ]:
# STORE = False
STORE = True

In [ ]:
df = pd.read_csv("data/runs-23_06_46-17-Apr-25.csv")

In [ ]:
cv_runs = [name for name in df.experiment if name.split("_")[-1].split("-")[0] == "fold"]
print("# models:", len(cv_runs) / 3)

In [ ]:
df_cv = df[df['experiment'].isin(cv_runs)]
df_cv = df_cv[["experiment", 'AUC_val', 'AUC_train', 'training_time_min', 'avg_epoch_time', 'nr_params', 'nr_flops', "best_epoch", "model_config.model"]]
df_cv["model_config.model"] = ["baseline_freq" if "freq" in name else model.strip("\"") for name, model in zip(df_cv["experiment"], df_cv["model_config.model"])]
df_cv = df_cv.rename(columns={'model_config.model': 'model', "avg_epoch_time": "min_per_epoch"})
df_cv = df_cv.drop("experiment", axis=1)
cols_to_convert = [col for col in df_cv.columns if col != 'model']
df_cv[cols_to_convert] = df_cv[cols_to_convert].astype(float)

# custom colnames
df_cv.columns = ["val AUC", "train AUC", "train time (min.)", "min. per epoch", "# parameters", "# flops", "best epoch", "model"]

# add rfr baseline results as separate runs
# RandomForestClassifier
# Mean Test ROC: 0.6511249520403964 0.006805343628861114
# {'fit_time': array([7.19223166, 7.37786293, 7.24737144]), 'score_time': array([0.31503367, 0.10961747, 0.10889411]), 'test_roc_auc': array([0.6419549 , 0.65317981, 0.65824014]), 'train_roc_auc': array([1., 1., 1.])}

rfr_baseline = pd.DataFrame({
    "val AUC": [0.6419549 , 0.65317981, 0.65824014],
    "train AUC": [1] * 3,
    "train time (min.)": [7.19223166 / 60, 7.37786293 / 60, 7.24737144 / 60],
    "min. per epoch": [np.nan] * 3,
    "# parameters": [np.nan] * 3,
    "# flops": [np.nan] * 3,
    "best epoch": [np.nan] * 3,
    "model": ["RFR (freq)"] * 3
})

df_cv = pd.concat([df_cv, rfr_baseline], axis=0)
df_cv.reset_index(drop=True, inplace=True)

In [ ]:
df_cv

In [ ]:
df_grouped = df_cv.groupby("model").mean()
df_grouped

In [ ]:
df_grouped_std = df_cv.groupby("model").std()
df_grouped_std.columns = [f"{col} std" for col in df_grouped_std.columns]
df_grouped_std

In [ ]:
# add all information
df_grouped = pd.concat([df_grouped, df_grouped_std], axis=1)

# change model names
df_grouped = df_grouped.rename(index={
    "baseline": "Baseline MLP",
    "baseline_freq": "Baseline MLP (freq)",
    "cnn": "CNN",
    "gru": "GRU",
    "lstm": "LSTM",
    "mamba": "Mamba",
    "transformer": "Transformer",
    "xlstm": "xLSTM",
    "LEGnet": "LEGnet",
    "RFC (freq)": "RFC (freq)"
})

## Results barplots

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import t

def plot_variable(df_plot, var, x_min=0, log_scale=False, use_sem=False, use_ci=False, store=False):
    # Sort dataframe by the variable to plot
    df_plot = df_plot.sort_values(var)
    df_plot.sort_values(by=var, inplace=True)
    
    # Compute uncertainty (SEM or SD)
    if use_ci:
        N = 3  # Number of CV runs
        df = N - 1  # Degrees of freedom
        t_critical = t.ppf(0.975, df)  # t-value for 95% confidence interval
        uncertainty = df_plot[f"{var} std"] / np.sqrt(N) * t_critical
    elif use_sem:
        uncertainty = df_plot[f"{var} std"] / np.sqrt(3)
    else:
        uncertainty = df_plot[f"{var} std"]
    
    # Plot the data with error bars
    plt.figure(figsize=(7, 3.5))
    if use_ci or use_sem:
        #plt.errorbar(df_plot[var], df_plot.index, xerr=uncertainty, fmt='o', color='skyblue', ecolor='lightgray', elinewidth=2, capsize=0)
        plt.barh(df_plot.index, df_plot[var], xerr=uncertainty, color='skyblue', capsize=2, ecolor="gray")
        plt.title(f"Mean {var} \n (with uncertainty over 3 folds)")
    else:
        plt.barh(df_plot.index, df_plot[var], xerr=uncertainty, color='skyblue')
        plt.title(f"Mean {var} \n (over 3 folds)")
    plt.xlabel(var)
    
    # Set x-axis limits and scaling
    if x_min:
        plt.xlim(x_min, df_plot[var].max() * 1.03)
    if log_scale:
        plt.xscale('log')

    if store:
        var_name = var.replace(" ", "_")
        plt.savefig(os.getenv("OUTPUT_DIR") + f"/cv_results_{var_name}.pdf", format="pdf", bbox_inches="tight")

    # Display the plot
    plt.show()

In [ ]:
plot_variable(df_grouped.copy(), "val AUC", x_min=0.5, use_sem=True, store=STORE)

In [ ]:
plot_variable(df_grouped.copy(), "train time (min.)", log_scale=True, use_sem=True)

In [ ]:
plot_variable(df_grouped.copy(), "# parameters", log_scale=True)

In [ ]:
plot_variable(df_grouped.copy(), "# flops", log_scale=True)

## Results Table

In [ ]:
df_table = df_grouped.copy()
df_table.sort_values(by="val AUC", ascending=False, inplace=True)
df_table.reset_index(inplace=True)
df_table = df_table.round(5)
#df_table.drop(["avg_epoch_time"], axis=1, inplace=True)
df_table[["# parameters", "# flops"]] = df_table[["# parameters", "# flops"]].astype(pd.Int64Dtype())

In [ ]:
df_table

In [ ]:
df_table.columns

In [ ]:
df_table = df_table[['model', 'val AUC', 'train AUC', 'train time (min.)',
                     '# parameters', '# flops', 'min. per epoch', 'best epoch']]
df_table.columns = ['model', 'mean val AUC', ' mean train AUC', 'mean train time (min.)', 'mean # parameters',
                    '# flops', 'min. per epoch', 'best epoch']

In [ ]:
def transparent_nan(val):
    if pd.isnull(val) or val is pd.NA:
        return 'background-color: white; color: white;'
    return ''

styled_df = (
    df_table.style
    .background_gradient(subset=['mean val AUC'], cmap='Greens')  # Color scale for 'val AUC'
    .background_gradient(subset=['mean train time (min.)'], cmap='Reds')  # Color scale for 'train time'
    .background_gradient(subset=['mean # parameters'], cmap='Reds')  # Color scale for '# Parameters'
    .background_gradient(subset=['# flops'], cmap='Reds')  # Color scale for '# Parameters'
    .background_gradient(subset=['min. per epoch'], cmap='Reds')  # Color scale for '# Parameters'
    .background_gradient(subset=['best epoch'], cmap='Reds')  # Color scale for '# Parameters'
    .map(transparent_nan)
    .format(precision=4)
)

styled_df

In [ ]:
# FIXME
if STORE:
    from html2image import Html2Image

    # write to html
    html_path = os.path.join(os.getenv("OUTPUT_DIR"), "cv_results.html")
    with open(html_path, "w", encoding="utf-8") as f:
        f.write(styled_df.to_html())

    html_path = os.path.join(os.getenv("OUTPUT_DIR"), "cv_results.html")
    hti = Html2Image(output_path=os.getenv("OUTPUT_DIR"))
    hti.screenshot(html_file=html_path, save_as="cv_results.png")#, size=(1000, 500))

In [ ]:
# TESTING
hti = Html2Image(output_path=os.getenv("OUTPUT_DIR"), browser="edge", size=(500, 200))
hti.screenshot(url='https://www.python.org', save_as='python_org.png')